# ShipChecker

## Configuration

In [41]:
## raw data path
rawDataPath = './portData/rawData.txt'

## google search api path
googleSearchApiPath = '../googleSearchApi.txt'
## google search custom search engine id
googleSearchCseIdPath = '../googleSearchCSE.txt'

#Processingdata path
processingDataPath = './portData/processingData.csv'

In [25]:
import pandas as pd
import re
from googleapiclient.discovery import build

## Load data

In [10]:
# load raw data and show the first 5 rows
rawData = pd.read_csv(rawDataPath, sep='\t')
rawData.head(5)

,DATE & TIME,ARRIVAL / DEPARTURE,VESSEL,VESSEL TYPE,FROM,TO,IN PORT
0,Wed 20 Mar 20:00,ARRIVAL,KOTA LIMA,CONTAINER SHIP (FULLY CELLULAR),CHIWAN PT,12 BROTHERSON DOCK (BD12),NO
1,Wed 20 Mar 22:00,ARRIVAL,MSC KANU F,CONTAINER SHIP (FULLY CELLULAR),BELL BAY,9 BROTHERSON DOCK (BD9),NO
2,Thu 21 Mar 03:00,ARRIVAL,WIDE INDIA,CONTAINER SHIP (FULLY CELLULAR),AUCKLAND,7 BROTHERSON DOCK (BD7),NO
3,Thu 21 Mar 15:00,ARRIVAL,GASCHEM HOMER,LPG TANKER,BRISBANE,BULK LIQUID BERTH 1 (BLB1),NO
4,Thu 21 Mar 17:00,ARRIVAL,MARI COUVA,CHEMICAL/PRODUCTS TANKER,ONSAN/ULSAN,BULK LIQUID BERTH 1 (BLB1),NO


In [11]:
# load google search api key and custom search engine id
googleApiKey = open(googleSearchApiPath, 'r').read()
print(googleApiKey[:5])
googleSearchCseId = open(googleSearchCseIdPath, 'r').read()
print(googleSearchCseId)

AIzaS
e0f84b16165d14dc2


In [37]:
# google search function and data extraction

api_key = googleApiKey
cse_id = googleSearchCseId

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

def get_link(results):
    link = results['items'][0]['link']
    return link
    
def get_snippet(results):
    snippet = results['items'][0]['snippet']
    return snippet

def get_IMO_number(snippet):
    imo_number_match = re.search(r'IMO number, (\d+)', snippet)
    if imo_number_match:
        imo_number = imo_number_match.group(1)
    else:
        imo_number = None  
    return imo_number

def get_vessel_details(vessel_name):
    search_query = f"what is imo number for {vessel_name}"
    results = google_search(search_query, api_key, cse_id)
    link = get_link(results)
    snippet = get_snippet(results)
    imo_number = get_IMO_number(snippet)
    
    return link, snippet, imo_number

In [39]:
processingData = rawData.head(1)

processingData[['link','snippet','imo_number']] = processingData.apply(lambda row: get_vessel_details(row['VESSEL']), axis=1, result_type='expand')

processingData.to_csv(processingDataPath)

C:\Users\admin\AppData\Local\Temp\ipykernel_10720\2817326180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processingData[['link','snippet','imo_number']] = processingData.apply(lambda row: get_vessel_details(row['VESSEL']), axis=1, result_type='expand')
C:\Users\admin\AppData\Local\Temp\ipykernel_10720\2817326180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processingData[['link','snippet','imo_number']] = processingData.apply(lambda row: get_vessel_details(row['VESSEL']), axis=1, result_type=

,DATE & TIME,ARRIVAL / DEPARTURE,VESSEL,VESSEL TYPE,FROM,TO,IN PORT,link,imo_number,snippet
0,Wed 20 Mar 20:00,ARRIVAL,KOTA LIMA,CONTAINER SHIP (FULLY CELLULAR),CHIWAN PT,12 BROTHERSON DOCK (BD12),NO,https://www.vesselfinder.com/vessels/details/9...,9267651,"IMO number, 9267651. Vessel Name, KOTA LIMA. S..."
